# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
keras = tf.keras
from sklearn.preprocessing import RobustScaler
import plotly.graph_objects as go
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Importing Data

In [ ]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# Data Pre-Processing

In [ ]:
death_df = death_df.groupby("Country/Region").sum().reset_index()
Country_death =  death_df['Country/Region'].values
death_df = death_df.drop(['Lat','Long','Country/Region'], axis=1)
death_df = death_df[death_df.columns].T
death_df.columns = Country_death
Date_index = death_df.index
death_df = death_df.diff().fillna(0)
death_df = death_df.abs()
death_df.head()

# Scalling the data

In [ ]:
sc = RobustScaler()
death_df_scaled = sc.fit_transform(death_df)
death_df_scaled = pd.DataFrame(death_df_scaled)

# Enter the number of days you want to predict

In [ ]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))
predict_death_index = pd.date_range(start=Date_index[-1],periods=DAYS_TO_PREDICT + 1,closed='right')
predict_death_index = predict_death_index.strftime('%Y/%m/%d')

# Creating class for building model

In [ ]:
class Corona_death_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')
        
        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
        
        
        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        death_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])
               
        optimizer = keras.optimizers.Adadelta(lr = 0.001)
        
        death_model.compile(loss=['mae'],optimizer=optimizer)   
                
        death_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10,verbose = 0)

        death_test_seq  = trainX[-1:]
        death_test_seq = death_test_seq
        death_test_seq = death_test_seq.reshape(len(death_test_seq), time_steps, 1)
        death_preds = []
        
        for _ in range (DAYS_TO_PREDICT):
            death_pred = death_model.predict(death_test_seq)
            death_preds.append(death_pred)
            death_new_seq = death_pred
            death_test_seq = death_new_seq
            death_test_seq = death_test_seq.reshape(len(death_test_seq ), time_steps, 1)

        

        return death_preds

# Calling the class and predicting the cases for every individual country

In [ ]:
NN = Corona_death_model()
death_predictions = []
for i in range(len(Country_death)):
    result = NN.Model(death_df_scaled,i) 
    death_predictions.append(result)

In [ ]:
death_predictions = (np.array(death_predictions).transpose())
death_predictions = death_predictions.reshape(((DAYS_TO_PREDICT),len(Country_death)))
death_predictions = sc.inverse_transform(death_predictions)
death_predictions = pd.DataFrame(death_predictions)
death_predictions = death_predictions.abs()
death_predictions['Dates'] = predict_death_index
death_predictions.set_index('Dates', inplace = True)
death_predictions.columns = Country_death
death_predictions.head()

# Visualizing The Death Cases For A Country As Per The User's Choice

### List of Infected Countries

In [ ]:
Infected_Countries = pd.DataFrame(Country_death)
Infected_Countries.columns = ['Countries']
pd.set_option("max_rows", None)
Infected_Countries

### Select The Country

In [ ]:
Country = input('Enter the name of the country from above list you would like to see the daily cases till now : ')

## Till Date Daily Cases of Death due to coronavirus

In [ ]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Death Daily Cases','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                                    xaxis_title="Dates",
                                    yaxis_title="Number Of Cases")
fig.add_trace(go.Scatter(x=death_df.index, 
                         y=death_df[Country],
                         mode='lines+markers',
                         line=dict(color='red', width=2)))

## Predictions of death cases due to coronavirus

In [ ]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Death Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},xaxis_title="Dates",yaxis_title="Number Of Cases")                           
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=death_predictions.index, 
                         y=death_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='turquoise', width=2)))